In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
from requests.structures import CaseInsensitiveDict
import os

# Import API key
from api_keys import geoapify_key

# Load the CSV file
tesla_ev_wa_df = pd.read_csv("resources/tesla_ev_wa.csv")

# Display
tesla_ev_wa_df.head()

,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Groups With Access Code,Access Days Time,EV Network,Facility Type,EV Pricing,Geocode Status,Latitude,Longitude,Status Code
0,ELEC,"Aberdeen, WA - Tesla Supercharger",416 Wishkah Street,Aberdeen,WA,98520,Public,24 hours daily,Tesla,NaN,NaN,GPS,46.976679,-123.813485,E
1,ELEC,Angel of the Winds Casino - Tesla Supercharger,3438 Stoluckquamish Lane,Arlington,WA,98223,Public,24 hours daily,Tesla,OTHER_ENTERTAINMENT,NaN,GPS,48.212441,-122.183491,E
2,ELEC,Fairfield Inn & Suites Burlington - Tesla Supe...,9384 Old Highway 99 North,Burlington,WA,98233,Public,24 hours daily,Tesla,NaN,NaN,200-8,48.509743,-122.338681,E
3,ELEC,Centralia Outlets - Tesla Supercharger,1200 Lum Rd,Centralia,WA,98531,Public,24 hours daily,Tesla,NaN,NaN,200-9,46.729872,-122.977392,E
4,ELEC,Holiday Inn Express - Tesla Supercharger,1620 Canyon Rd,Ellensburg,WA,98926,Public,24 hours daily,Tesla,NaN,NaN,GPS,46.976918,-120.541620,E


In [1]:
# Create a map
map = folium.Map(location=[47.7511, -120.7401], zoom_start=7)

# Add markers for each address in the DataFrame
for index, row in tesla_ev_wa_df.iterrows():
    # Construct popup content with additional details
    popup_content = f"<b>Station Name:</b> {row['Station Name']}<br>"
    popup_content += f"<b>Street Address:</b> {row['Street Address']}<br>"
    popup_content += f"<b>City:</b> {row['City']}<br>"
    popup_content += f"<b>EV Network:</b> {row['EV Network']}<br>"

    # Add a marker for each address using latitude and longitude
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_content,
        icon='home' 
    ).add_to(map)

# Display the map
map

NameError: name 'folium' is not defined

In [6]:
# Create an empty list to store coffee information
coffee_info = []

# Set parameters to search for restaurants
limit = 2     # Limit the number of results to 2
radius_meters = 750 # 1/2 mile
# Print a message to indicate the start of the restaurant search
print("Starting coffee search")

# Iterate through the tesla_ev_wa_df DataFrame
for index, row in tesla_ev_wa_df.iterrows():
    # Construct the API request URL
    base_url = "https://api.geoapify.com/v2/places"
    url = f"{base_url}?categories=catering.cafe.coffee_shop&lat={row['Latitude']}&lon={row['Longitude']}&radius={radius_meters}&limit={limit}&apiKey={geoapify_key}"

    # Set headers
    headers = {
        "Accept": "application/json"
    }
    
    try:
        # Make an API request using the URL
        resp = requests.get(url, headers=headers)
        
        # Convert the API response to JSON format
        coffee_data = resp.json()

        # Extract coffee information from the response
        if "features" in coffee_data and coffee_data["features"]:
            for restaurant in coffee_data["features"]:
                name = restaurant["properties"]["name"]
                latitude = restaurant["geometry"]["coordinates"][1]
                longitude = restaurant["geometry"]["coordinates"][0]
                formatted_address = restaurant["properties"]["formatted"]
                
                coffee_info.append({
                    "Address": row['Street Address'],
                    "City": row['City'],
                    "State": row['State'],
                    "ZIP": row['ZIP'],
                    "Restaurant Name": name,
                    "Latitude": latitude,
                    "Longitude": longitude,
                    "Formatted Address": formatted_address
                })
        else:
            coffee_info.append({
                "Address": row['Street Address'],
                "City": row['City'],
                "State": row['State'],
                "ZIP": row['ZIP'],
                "Restaurant Name": "",
                "Latitude": row['Latitude'],
                "Longitude": row['Longitude'],
                "Formatted Address": ""
            })
        
    except Exception as e:
        print(f"Error occurred while searching for coffee near {row['Street Address']}, {row['City']}, {row['State']} {row['ZIP']}: {e}")

# Convert the list of coffee information to a DataFrame
coffee_info_df = pd.DataFrame(coffee_info)

# Save the DataFrame to a CSV file in the resources folder
coffee_info_df.to_csv("resources/coffee_info_tesla_ev.csv", index=False)

# Print a message indicating that the CSV file has been saved
print("CSV file saved: coffee_info_tesla_ev.csv")


Starting restaurant search
Error occurred while searching for restaurants near 3438 Stoluckquamish Lane, Arlington, WA 98223: 'name'
Error occurred while searching for restaurants near 3438 Stoluckquamish Ln, Arlington, WA 98223: 'name'
Error occurred while searching for restaurants near 3600 Suncadia Trl, Cle Elum, WA 98922: 'name'
Error occurred while searching for restaurants near 15015 E Indiana Ave, Spokane Valley, WA 99216: 'name'
Error occurred while searching for restaurants near 604 Patterson Lake Rd, Winthrop, WA 98862: 'name'
Error occurred while searching for restaurants near 811 WA-970, Cle Elum, WA 98922: 'name'
Error occurred while searching for restaurants near 10506 West Aero Road, Spokane, WA 99224: 'name'
Error occurred while searching for restaurants near 11616 E Montgomery Dr, Spokane Valley, WA 99206: 'name'
Error occurred while searching for restaurants near 5219  Tacoma Mall Boulevard, Tacoma, WA 98409: 'name'
Error occurred while searching for restaurants near 

In [7]:
coffee_info_df.head()

,Address,City,State,ZIP,Restaurant Name,Latitude,Longitude,Formatted Address
0,416 Wishkah Street,Aberdeen,WA,98520,Dutch Bros. Coffee,46.977415,-123.808426,"Dutch Bros. Coffee, East Wishkah Street, Aberd..."
1,416 Wishkah Street,Aberdeen,WA,98520,Starbucks,46.977603,-123.802957,"Starbucks, 1202 East Wishkah Street, Aberdeen,..."
2,9384 Old Highway 99 North,Burlington,WA,98233,The Coffee Barn,48.507100,-122.337733,"The Coffee Barn, 9552 Old Highway 99 North Roa..."
3,9384 Old Highway 99 North,Burlington,WA,98233,Foxy Lady Latte,48.506708,-122.337614,"Foxy Lady Latte, 9566 Old Highway 99 North Roa..."
4,1200 Lum Rd,Centralia,WA,98531,Starbucks,46.728547,-122.981222,"Starbucks, Harrison Avenue, Centralia, WA 9853..."
